# 実験概要
## CMNによる特徴量抽出
**ケプストラム平均正規化** (Cepstrum Mean Normalization : CMN) を使用し、特徴量を抽出する。
切り出した波形に対して**離散フーリエ変換** (Discrete Fourier Transform : DFT) を行い、絶対値を取ることで**振幅スペクトル**を得る。
これに対して対数を取って**対数振幅スペクトル**に変換し、逆離散フーリエ変換 (Inverse Discrete Fourier Transform : IDFT) を行うことで、ケプストラム領域へと変換し、先頭50要素を切り出し、結合した100要素の配列を前処理として返す。
この前処理は
```
left, right, posture = pp.slicer("raw\\" + tester.**.**.value)
cepstrum = pp.cmn_denoise(left, right)
```
によって行われる。

## DNNによる寝姿勢分類
DNNによって前処理したデータから学習・分類を行う。

# モジュールのインポート

In [2]:
import numpy as np
import glob
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 自作モジュール
import data_path as dpath
import preprocess as pp

# DataLoaderの定義

In [18]:
# tester以外のrawを読み込む
for p in glob.glob(".\\raw\\LMH\\*\\", recursive=True):
    print(p)

In [21]:
# 学習用データセットを読み込むためのDataLoaderを定義
class train_datasets(Dataset):
    def __init__(self, type, mat, tester, transforms=None):
        self.type = type
        self.mat = mat
        self.tester = tester
        self.transform = transforms

        # tester以外のrawを読み込む
        self.train_cep = np.empty((0, 100))  # 100要素の配列
        self.train_posture = np.empty(0) # 姿勢データ配列
        self.tester_path = dpath.get_path(type, mat, tester)
        for p in glob.glob(".\\raw\\LMH\\*\\", recursive=True):
            for tpath in self.tester_path:
                if p == tpath:
                    continue
            left, right, posture = pp.slicer(p)
            cep = pp.cmn_denoise(left, right)
            self.train_cep = np.vstack(self.train_cep, cep)
            self.train_posture = np.hstack(posture)
        print(self.train_cep.shape, self.train_posture.shape)

    def __len__(self):
        return len(self.train_datasets)

    def __getitem__(self, idx):
        return self.train_datasets[idx]

class test_datasets(Dataset):
    def __init__(self, tester, transform=None):
        self.tester = tester.value
        self.transform = transform

        # 前処理したrawを読み込む
        self.left, self.right, self.posture = pp.slicer("raw\\"+self.csv_file)
        self.cepstrum = pp.cmn_denoise(self.left, self.right)

    def __len__(self):
        return len(self.cepstrum)

    def __getitem__(self, idx):
        return self.cepstrum[idx], self.posture[idx]